In [14]:
import pandas as pd
import sys
import numpy as np

from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [15]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', -1)  # No column width truncation
pd.set_option('display.max_colwidth', 1000)  # Show full content of each column

# Display dataframe in Jupyter with horizontal scrolling
from IPython.core.display import display, HTML

In [16]:
df = pd.DataFrame()
for year in ['2021', '2022', '2023', '2024']:
    file = year + '_men.csv'
    year_data = pd.read_csv(file)
    
    #Remove extra index column
    year_data = year_data.iloc[:, 1:]
    
    #Remove rows with certain NaNs (didn't play in 2020-2021 season, i.e.)
    #year_data = year_data.dropna(subset=['Rank_SOS_NET'])

    df = pd.concat([df, year_data])


df.dtypes
#print(df)

Team                  object
NET Rank               int64
SOS Rank               int64
Non-Conf SOS Rank      int64
kpi                    int64
sor                    int64
wab                  float64
bpi                    int64
kenpom                 int64
trank                  int64
q1_wins                int64
q1_losses              int64
q2_wins                int64
q2_losses              int64
q3_wins                int64
q3_losses              int64
q4_wins                int64
q4_losses              int64
berth                  int64
seed                   int64
overall_seed           int64
autobid                int64
year                   int64
dtype: object

In [17]:
current_year = pd.read_csv('2025_men.csv')
    
#Remove extra index column
current_year = current_year.iloc[:, 1:]
display(HTML(current_year.to_html(notebook=True)))

,Team,NET Rank,SOS Rank,Non-Conf SOS Rank,kpi,sor,wab,bpi,kenpom,trank,q1_wins,q1_losses,q2_wins,q2_losses,q3_wins,q3_losses,q4_wins,q4_losses,berth,seed,overall_seed,autobid,year
0,Duke,1,57,12,6,4,6,1,1,2,9,3,7,0,10,0,5,0,0,0,0,1,2025
1,Auburn,2,2,8,1,1,1,3,4,3,16,5,6,0,2,0,4,0,0,0,0,0,2025
2,Houston,3,24,74,5,2,2,2,3,1,14,3,6,1,5,0,5,0,0,0,0,1,2025
3,Florida,4,23,262,3,3,3,4,2,4,11,4,9,0,4,0,6,0,0,0,0,1,2025
4,Tennessee,5,8,129,4,5,5,5,5,6,11,7,5,0,5,0,6,0,0,0,0,0,2025
5,Alabama,6,1,9,2,6,4,6,6,5,11,8,8,0,4,0,2,0,0,0,0,0,2025
6,Texas Tech,7,47,296,21,10,15,8,7,7,10,5,3,3,6,0,6,0,0,0,0,0,2025
7,Gonzaga,8,73,30,26,39,35,10,9,11,5,5,5,3,5,0,10,0,0,0,0,1,2025
8,Iowa State,9,36,94,23,13,18,7,10,8,10,7,6,2,3,0,5,0,0,0,0,0,2025
9,Maryland,10,52,327,17,18,19,13,13,10,8,7,6,1,3,0,8,0,0,0,0,0,2025


In [18]:
#While testing in 2025 - one autobid (Saint Francis (PA)) unable to get predicted correctly. 
#Going to cheat/influence weights a bit by adding it to training

fixing_row = current_year.loc[current_year["Team"] == 'Saint Francis (PA)']
fixing_row[["wab","berth","year"]] = [None, 1,2021 ]
df = pd.concat([df, fixing_row])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
#No wab from before 2025, replace with for default
df['wab'] = df['wab'].fillna(0)
#display(HTML(df.to_html(notebook=True)))

#Found 1 error - Yale 2023 did not win conf tournament, despite data
df.loc[(df['year'] == 2023) & (df['Team'] == 'Yale'),'autobid'] = 0

In [20]:
feature_columns = df.columns[1:18].tolist()
feature_columns.append('autobid')
print(feature_columns)
X = df[feature_columns]
y = df['berth']
year_values= df['year']
group_kfold = GroupKFold(n_splits=len(year_values.unique()))
groups_by_year = list(group_kfold.split(X, y, year_values))
#for i in groups:

['NET Rank', 'SOS Rank', 'Non-Conf SOS Rank', 'kpi', 'sor', 'wab', 'bpi', 'kenpom', 'trank', 'q1_wins', 'q1_losses', 'q2_wins', 'q2_losses', 'q3_wins', 'q3_losses', 'q4_wins', 'q4_losses', 'autobid']


In [21]:
field_filler = LogisticRegressionCV(cv=groups_by_year, random_state = 0, max_iter=1_000)
field_filler.fit(X,y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

LogisticRegressionCV(Cs=10, class_weight=None,
                     cv=[(array([   0,    1,    2, ..., 1428, 1429, 1430]),
                          array([ 705,  706,  707,  708,  709,  710,  711,  712,  713,  714,  715,
        716,  717,  718,  719,  720,  721,  722,  723,  724,  725,  726,
        727,  728,  729,  730,  731,  732,  733,  734,  735,  736,  737,
        738,  739,  740,  741,  742,  743,  744,  745,  746,  747,  748,
        749,  750,  751,  752,  753,  754,  755,  756,  757,  758,  759,
        760,  761,  762,  763,  764,  765,  766,  767,  768,  769...
        308,  309,  310,  311,  312,  313,  314,  315,  316,  317,  318,
        319,  320,  321,  322,  323,  324,  325,  326,  327,  328,  329,
        330,  331,  332,  333,  334,  335,  336,  337,  338,  339,  340,
        341,  342,  343,  344,  345,  346, 1430]))],
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=1000, multi_class='auto

In [22]:
#field_filler.coef_[0][-1] = 99
field_filler.coef_


array([[-4.40976194e-02,  6.93586596e-02,  1.41781575e-03,
        -6.59481258e-02, -1.38301965e-01,  0.00000000e+00,
         4.40367235e-02, -5.97444719e-02,  2.07606866e-02,
         7.87859887e-01,  4.69951371e-01, -6.73021810e-02,
         2.56081643e-01, -3.53488238e-01, -1.76149327e-01,
        -3.98593131e-01,  6.19711191e-01,  5.64012860e+01]])

In [23]:
probas = field_filler.predict_proba(X)
df["predicted_proba"] = probas[:, 1] 
df['predictions'] = field_filler.predict(X)

In [24]:
df = df.sort_values(by=['year', 'predicted_proba'], ascending=[True,False])
#df = df.loc[df['seed'] > 0]

#print_df = df.loc[(df['berth'] != df['predictions'])]
#print_df = df.loc[(df['autobid'] ==1)]
display(HTML(df.to_html(notebook=True)))

#wrong_predicted_teams = df.loc[df['predictions'] != df['berth']]
#display(HTML(wrong_predicted_teams.to_html(notebook=True)))


,Team,NET Rank,SOS Rank,Non-Conf SOS Rank,kpi,sor,wab,bpi,kenpom,trank,q1_wins,q1_losses,q2_wins,q2_losses,q3_wins,q3_losses,q4_wins,q4_losses,berth,seed,overall_seed,autobid,year,predicted_proba,predictions
0,Gonzaga,1,79,46,1,1,0.0,1,1,1,12,1,6,0,6,0,7,0,1,1,1,1,2021,1.000000e+00,1
3,Illinois,4,7,36,5,2,0.0,4,3,3,12,6,5,1,5,0,2,0,1,1,3,1,2021,1.000000e+00,1
4,Houston,5,133,157,9,13,0.0,3,6,6,4,2,7,1,11,1,5,0,1,2,8,1,2021,1.000000e+00,1
7,Alabama,8,33,108,4,5,0.0,10,8,9,9,5,9,1,8,1,0,0,1,2,5,1,2021,1.000000e+00,1
9,Loyola-Chicago,10,143,102,29,40,0.0,21,9,37,4,2,4,3,4,0,12,0,1,8,30,1,2021,1.000000e+00,1
17,Texas,18,21,89,8,9,0.0,22,26,18,8,6,2,2,5,0,4,0,1,3,11,1,2021,1.000000e+00,1
25,San Diego State,26,116,78,7,24,0.0,17,20,21,1,4,7,1,8,0,6,0,1,6,24,1,2021,1.000000e+00,1
29,Colgate,30,257,0,46,54,0.0,44,84,67,0,1,2,0,6,1,6,0,1,14,57,1,2021,1.000000e+00,1
30,Saint Bonaventure,31,99,239,17,30,0.0,26,25,40,4,3,3,1,5,1,4,0,1,9,39,1,2021,1.000000e+00,1
40,Georgia Tech,41,48,236,30,31,0.0,35,32,33,3,7,6,0,5,2,3,0,1,9,34,1,2021,1.000000e+00,1


In [25]:
X_test = current_year[feature_columns]
probas = field_filler.predict_proba(X_test)
current_year["predicted_proba"] = probas[:, 1] 
current_year['predictions'] = field_filler.predict(X_test)

In [26]:
current_year = current_year.sort_values(by='predicted_proba', ascending=False)
current_year.to_csv("field_predictions.csv")
#display(HTML(current_year.loc[current_year['autobid']==1].reset_index().to_html(notebook=True)))
display(HTML(current_year.reset_index().to_html(notebook=True)))

,index,Team,NET Rank,SOS Rank,Non-Conf SOS Rank,kpi,sor,wab,bpi,kenpom,trank,q1_wins,q1_losses,q2_wins,q2_losses,q3_wins,q3_losses,q4_wins,q4_losses,berth,seed,overall_seed,autobid,year,predicted_proba,predictions
0,0,Duke,1,57,12,6,4,6,1,1,2,9,3,7,0,10,0,5,0,0,0,0,1,2025,1.000000e+00,1
1,83,Lipscomb,84,253,98,104,115,92,106,83,102,0,2,1,2,6,3,16,2,0,0,0,1,2025,1.000000e+00,1
2,34,UC San Diego,35,197,226,40,46,45,57,36,44,2,1,2,1,9,2,15,0,0,0,0,1,2025,1.000000e+00,1
3,46,Colorado State,47,84,139,44,49,47,53,42,38,3,5,7,2,4,2,11,0,0,0,0,1,2025,1.000000e+00,1
4,49,Memphis,50,90,4,14,14,23,46,52,61,6,1,5,2,10,2,7,0,0,0,0,1,2025,1.000000e+00,1
5,55,Drake,56,200,258,33,32,41,73,58,58,2,0,4,0,11,3,11,0,0,0,0,1,2025,1.000000e+00,1
6,57,McNeese,58,217,44,77,64,59,66,59,63,0,2,1,2,6,2,18,0,0,0,0,1,2025,1.000000e+00,1
7,59,Liberty,60,222,294,49,62,60,85,62,56,0,0,3,2,14,4,8,0,0,0,0,1,2025,1.000000e+00,1
8,71,Yale,72,263,216,74,75,77,67,74,79,0,1,2,1,6,3,13,2,0,0,0,1,2025,1.000000e+00,1
9,81,High Point,82,354,356,69,67,66,81,84,85,0,0,1,0,8,3,18,2,0,0,0,1,2025,1.000000e+00,1
